In [10]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [29]:
ENDPOINT = 'https://api.thegraph.com/subgraphs/name/convex-community/volume-%s-test'
networks = ['mainnet','matic', 'arbitrum', 'avalanche', 'fantom']
WEEK = 3600 * 24 * 7

In [12]:
def grt_query(query, network):
    r = requests.post(ENDPOINT % network, json={'query': query})
    return r.json()['data']

pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [13]:
fee_query = """
{ pools(first: 1000) {
  dailyPoolSnapshots(first: 1000 orderBy: timestamp orderDirection: desc) {
    timestamp
    totalDailyFeesUSD
  }
}
}
"""

In [52]:
def get_chain_data(network):
    data = grt_query(fee_query, network)['pools']
    c = [el for a in data for el in a['dailyPoolSnapshots']]
    df = pd.DataFrame(c)
    df['totalDailyFeesUSD'] = df['totalDailyFeesUSD'].astype(float)
    df['timestamp'] = df['timestamp'].apply(lambda x: ((int(x) // WEEK) * WEEK))
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
    df.columns = ['week', network]
    df = df.groupby('week').sum()
    return df

In [55]:
res = []
for network in networks:
    res.append(get_chain_data(network))

In [58]:
data = res[0]
for i in range(len(res)-1):
    data = pd.merge(data, res[i+1], how='outer',left_index=True,right_index=True)

In [59]:
subgraph_rev = data.fillna(0).sort_index()
subgraph_rev

,mainnet,matic,arbitrum,avalanche,fantom
week,,,,,
2020-10-29,139380.2495,0.0000,0.0000,0.0000,0.0000
2020-11-05,295287.0187,0.0000,0.0000,0.0000,0.0000
2020-11-12,478040.1544,0.0000,0.0000,0.0000,0.0000
2020-11-19,366542.0986,0.0000,0.0000,0.0000,0.0000
2020-11-26,354029.4256,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...
2022-07-07,689522.6594,0.0000,18230.6699,20323.6754,13327.6837
2022-07-14,1061345.4180,0.0000,32504.3938,21242.8862,15541.5071
2022-07-21,931444.2791,0.0000,33567.8554,18211.3186,15567.4473


In [60]:
subgraph_rev.sum()

mainnet     213882711.0444
matic         9079348.1558
arbitrum      2360301.1539
avalanche     8412869.4064
fantom        5820400.6704
dtype: float64

In [61]:
subgraph_rev.sum().sum() / 2

119777815.21543603

,totalDailyFeesUSD
timestamp,
2020-10-29,139380.2495
2020-11-05,295287.0187
2020-11-12,478040.1544
2020-11-19,366542.0986
2020-11-26,354029.4256
...,...
2022-07-07,689522.6594
2022-07-14,1061345.4180
2022-07-21,931444.2791
